Old implementations sits here, just because

In [ ]:
class rowsList(BaseCli):
    def __init__(self, _slice):
        """Space-expensive implementation for :class:`rows`, without a lot
of flexibility. Just used for slices with negative start/stop really. Don't
use this directly, use :class:`rows` instead"""
        super().__init__(); self._slice = _slice; self.inverted = False
    def __ror__(self, it:Iterator[str]):
        it = list(it); full = range(len(it))
        rows = full[self._slice]
        if self.inverted: rows = [e for e in full if e not in rows]
        for row in rows: yield it[row]
    def __invert__(self): self.inverted = True; return self
class rows(BaseCli):
    def __init__(self, *rows:List[int]):
        """Cuts out specific rows. Space complexity O(1) as a list is not
constructed (unless you're using some really weird slices).

:param rows: ints for the row indices

Example::

    "0123456789" | rows(2) | deref() # returns ["2"]
    "0123456789" | rows(5, 8) | deref() # returns ["5", "8"]
    "0123456789" | rows()[2:5] | deref() # returns ["2", "3", "4"]
    "0123456789" | ~rows()[2:5] | deref() # returns ["0", "1", "5", "6", "7", "8", "9"]
    "0123456789" | ~rows()[:7:2] | deref() # returns ['1', '3', '5', '7', '8', '9']
    "0123456789" | rows()[:-4] | deref() # returns ['0', '1', '2', '3', '4', '5']
    "0123456789" | ~rows()[:-4] | deref() # returns ['6', '7', '8', '9']"""
        super().__init__()
        if len(rows) == 1 and isinstance(rows[0], slice):
            s = rows[0]
            start = s.start if s.start is not None else float("-inf")
            stop = s.stop if s.stop is not None else float("inf")
            self.domain = k1lib.Domain([start, stop])
            self.every = s.step or 1 # only used for slices really
        else:
            self.domain = k1lib.Domain.fromInts(*rows)
            self.every = 1
        self.inverted = False
    def _every(self, every): self.every = every; return self
    def __getitem__(self, _slice):
        s1, s2 = _slice.start, _slice.stop
        a = (_slice.start or 0) < 0; b = (_slice.stop or 0) < 0
        c = (_slice.step or 1)
        if a or b: # at least 1 is negative
            if c == 1:
                if b:
                    if s1 is None: return head(s2) # [None, -3]
                    else: return ~head(s1) | head(s2) # [5, -3]
                if a and s2 is None: return ~head(s1) # [-3, None]
                # else case is [-10, 6], which is weird, so just stick to the long one
            return rowsList(_slice) # worst case scenario
        answer = rows(_slice); answer.inverted = self.inverted; return answer
    def __invert__(self): self.inverted = not self.inverted; return self
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it)
        true, false = (False, True) if self.inverted else (True, False)
        def gates():
            gate = self.domain.intIter(self.every); x = 0
            for i in gate:
                while x < i: yield false; x += 1
                yield true; x += 1
            while True: yield false
        return (row for gate, row in zip(gates(), it) if gate)